# 예제 3.59: 다층 퍼셉트론 (Multi-Layer Perceptron, MLP)

## 학습목표
1. **다층 퍼셉트론(MLP)** 구조 이해하기 - 은닉층(Hidden Layer) 추가
2. **XOR 문제 해결** 방법 학습하기 - 다층 구조로 비선형 문제 해결
3. **은닉층의 역할** 파악하기 - 비선형 특성 학습
4. **단층 vs 다층 퍼셉트론** 비교하기

---

#### 라이브러리 임포트

**다층 퍼셉트론 (MLP)**
- 은닉층이 1개 이상 존재
- 비선형 분리 문제 해결 가능 (XOR 등)
- 각 층의 활성화 함수로 비선형성 도입

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

---

#### 데이터셋 및 모델 클래스 정의

**MLP 구조**: 입력(2) → 은닉층(2) → 출력(1)

In [ ]:
class CustomDataset(Dataset):
    """퍼셉트론용 데이터셋 (XOR 문제)"""
    
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.y = df.iloc[:, 2].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length


class CustomModel(nn.Module):
    """다층 퍼셉트론 (MLP) 모델"""
    
    def __init__(self):
        super().__init__()

        # 1층 (은닉층): 입력 2 → 출력 2
        self.layer1 = nn.Sequential(
            nn.Linear(2, 2),
            nn.Sigmoid()
        )
        # 2층 (출력층): 입력 2 → 출력 1
        self.layer2 = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer1(x)  # 은닉층 통과
        x = self.layer2(x)  # 출력층 통과
        return x

---

#### 데이터 및 모델 준비

In [ ]:
# 데이터셋 및 데이터로더 생성
# perceptron.csv: XOR 논리 게이트 데이터
train_dataset = CustomDataset("../datasets/perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

In [ ]:
# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 다층 퍼셉트론 모델 생성
model = CustomModel().to(device)

# 손실함수 및 옵티마이저
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

print("모델 구조:")
print(model)

---

#### 학습 루프

In [ ]:
# 학습 루프
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

---

#### 추론 (XOR 문제 테스트)

**XOR 진리표**
- (0, 0) → 0
- (0, 1) → 1
- (1, 0) → 1
- (1, 1) → 0

단층 퍼셉트론으로는 해결 불가, 다층 퍼셉트론으로 해결 가능!

In [ ]:
# 추론: XOR 테스트
with torch.no_grad():
    model.eval()
    
    # 테스트 입력: 가능한 모든 이진 조합
    inputs = torch.FloatTensor([
        [0, 0],  # XOR: 0
        [0, 1],  # XOR: 1
        [1, 0],  # XOR: 1
        [1, 1]   # XOR: 0
    ]).to(device)
    
    outputs = model(inputs)
    
    print("다층 퍼셉트론 (MLP) 결과:")
    print("---------")
    print("예측 확률:")
    print(outputs)
    print("\n예측 클래스 (임계값 0.5):")
    print((outputs > 0.5).int())
    print("\nXOR 정답: [0, 1, 1, 0]")